# Apache Arrow... in a nutshell

## Existing open standards
- XML, json
- SQL
- binary storage format with metadata (NetCDF, HDF5, Apache Parquet)
- serialization/ RPC protocols (Apache AVRO, protocol buffers)


## Why we need open standards?
Performance, no overhead, valid accross programming language

## Which benefits for Pandas?
Not based originally on open standards, that's it!

## Why columnar tables?
- SQL is row oriented format (ex. Apache Impala, PostgreSQL)
- but queries are often made on columns of a table, or on a subset of the columns.

## The Apache Arrow project
- **Goal**: Define an open standard for column-oriented tables (data frames) that is language-independant (Java, Python, R, Javascript, ...), so **portable accross languages**, **"zero-copy workflows"**
- need specifications, libraries, tools

> So you can have:
> - tables in-memory in JAVA
> - The Arrow-JAVA library can send "queries" to the Arrow-C++ librarie
> - hand off the data through J&I **without actually copying the data**  >>> **"zero-copy workflows"**
> - Arrow-C++ evaluate (compiles) the functions with LLVM
> - then send the data back throug the J&I bridge

## Coming if not there already
- Arrow for R
- Arrow on the GPU
- Data Access/Ingest
  - Apache Avro
  - Apache Parquet
  - Apache ORC
  - CSV
  - JSON
  - ODBC / JDBC
  - ...

## Resources

* Apache Parquet format: https://github.com/apache/parquet-format
* videos: 
  * 2018-07 by Wes McKinney: https://www.youtube.com/watch?v=y7zGnKzaKIw (existing standards, challenges)
  * 2019-06 by Wes McKinney: https://www.youtube.com/watch?v=uZA55cGDaBQ

# Reading and Writing the Apache Parquet Format
## Reading and Writing Single Files

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa

In [2]:

df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                  index=list('abc'))

df

,one,two,three
a,-1.0,foo,True
b,NaN,bar,False
c,2.5,baz,True


In [3]:
table = pa.Table.from_pandas(df)

In [4]:
table

pyarrow.Table
one: double
two: string
three: bool
__index_level_0__: string
metadata
--------
{b'pandas': b'{"index_columns": ["__index_level_0__"], "column_indexes": [{"na'
            b'me": null, "field_name": null, "pandas_type": "unicode", "numpy_'
            b'type": "object", "metadata": {"encoding": "UTF-8"}}], "columns":'
            b' [{"name": "one", "field_name": "one", "pandas_type": "float64",'
            b' "numpy_type": "float64", "metadata": null}, {"name": "two", "fi'
            b'eld_name": "two", "pandas_type": "unicode", "numpy_type": "objec'
            b't", "metadata": null}, {"name": "three", "field_name": "three", '
            b'"pandas_type": "bool", "numpy_type": "bool", "metadata": null}, '
            b'{"name": null, "field_name": "__index_level_0__", "pandas_type":'
            b' "unicode", "numpy_type": "object", "metadata": null}], "creator'
            b'": {"library": "pyarrow", "version": "0.14.1"}, "pandas_version"'
            b': "0.25.1"}'

In [5]:
# We write this to Parquet format with write_table:
import pyarrow.parquet as pq
pq.write_table(table, 'example.parquet')

In [6]:
# This creates a single Parquet file.
# In practice, a Parquet dataset may consist of many files
# in many directories.
!ls

demo_arrow.ipynb  example_noindex.parquet  example.parquet


In [7]:
# We can read a single file back with read_table:
table2 = pq.read_table('example.parquet')

In [8]:
table2.to_pandas()

,one,two,three
a,-1.0,foo,True
b,NaN,bar,False
c,2.5,baz,True


In [9]:
# You can pass a subset of columns to read,
# which can be much faster than reading the whole file
# (due to the columnar layout):
pq.read_table('example.parquet', columns=['one', 'three']).to_pandas()

,one,three
0,-1.0,True
1,NaN,False
2,2.5,True


In [10]:
# When reading a subset of columns from a file that used
# a Pandas dataframe as the source,
# we use read_pandas to maintain any additional index column data:
pq.read_pandas('example.parquet', columns=['two']).to_pandas()

,two
a,foo
b,bar
c,baz


## Omitting the DataFrame index

In [11]:
df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                  index=list('abc'))

df

,one,two,three
a,-1.0,foo,True
b,NaN,bar,False
c,2.5,baz,True


In [12]:
table = pa.Table.from_pandas(df, preserve_index=False)

In [13]:
pq.write_table(table, 'example_noindex.parquet')
t = pq.read_table('example_noindex.parquet')
t.to_pandas()
# Here you see the index did not survive the round trip

,one,two,three
0,-1.0,foo,True
1,NaN,bar,False
2,2.5,baz,True
